### To check the sample rate of the test recordings

In [2]:
import os
import pandas as pd
from opensoundscape.audio import Audio
from tqdm.autonotebook import tqdm

In [3]:
# Path to your test set directory
test_set_dir = "/mnt/d/retraining_BirdNET_2025/model_test/input_ready/"

In [5]:
# List to store results
sample_rate_data = []

# Loop through each audio file
for filename in os.listdir(test_set_dir):
    if filename.lower().endswith(".wav"):
        file_path = os.path.join(test_set_dir, filename)
        try:
            audio = Audio.from_file(file_path)
            sample_rate = audio.sample_rate
            sample_rate_data.append({
                "file": os.path.relpath(file_path, start=test_set_dir),
                "sample_rate": sample_rate
            })
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            sample_rate_data.append({
                "file": os.path.relpath(file_path, start=test_set_dir),
                "sample_rate": "ERROR"
            })

# Save to CSV
df = pd.DataFrame(sample_rate_data)


print("Sample rates saved to 'test_set_sample_rates.csv'")


Sample rates saved to 'test_set_sample_rates.csv'


In [8]:
# Export it as a .csv (optional)
#df.to_csv("/mnt/d/retraining_BirdNET_2025/model_test/input_ready/test_set_sample_rates.csv", index=False)

df.head(6)

,file,sample_rate
0,0.304_1_20220702_063000_rooster.wav,48000
1,0.388_1_20220712_053000_rooster.wav,48000
2,0.439_2_20220712_061000_rooster.wav,48000
3,0.577_2_20220703_045000_rooster.wav,48000
4,0.725_1_20220714_065000_rooster.wav,48000
5,0.815_3_20220706_033000_rooster.wav,48000


### I will resample the files that have a sample rate other than 48 kHz

In [ ]:
import os
import pandas as pd
from opensoundscape.audio import Audio
from tqdm import tqdm


In [10]:
# Load your CSV with sample rates
#csv_path = "/mnt/d/retraining_BirdNET_2025/model_test/input_ready/test_set_sample_rates.csv"
#df = pd.read_csv(csv_path)

# Base path of original audio files
base_dir = "/mnt/d/retraining_BirdNET_2025/model_test/input_ready"

# Output directory for resampled files
resample_dir = os.path.join(base_dir, "resampled_48kHz")

# Desired sample rate
target_sr = 48000

# Loop through the DataFrame and process only non-48kHz files
for _, row in tqdm(df.iterrows(), total=len(df), desc="Resampling"):
    file_rel_path = row['file']
    current_sr = row['sample_rate']
    
    # Skip files already at 48 kHz
    if current_sr == target_sr:
        continue

    input_path = os.path.join(base_dir, file_rel_path)
    output_path = os.path.join(resample_dir, file_rel_path)

    try:
        audio = Audio.from_file(input_path)
        audio.resample(target_sr)

        # Create output folder structure if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        audio.save(output_path)
    except Exception as e:
        print(f"Error resampling {input_path}: {e}")


Resampling:   0%|          | 0/208 [00:00<?, ?it/s]